In [1]:
import numpy as np
import sim
from utils import *
import networkx as nx
from tqdm import tqdm

In [2]:
# Get graph
G = get_graph_from_file("sample_graphs/J.5.1.json")

In [3]:
# Population
pop_size = 30
num_seeds = 10
num_parents = 4
num_mutations = int(num_seeds/3)
population = [np.random.choice(list(G.nodes), num_seeds, replace=False) for _ in range(pop_size)]

In [4]:
def calc_pop_fitness(pop, graph):
    fitness = np.zeros(pop_size)
    adj_list = nx.to_dict_of_lists(graph)
    for i in range(pop_size):
        for j in range(0, i, 1):
            # res = score_comp_seeds(list(pop[i]), list(pop[j]), adj_list)
            # if   res ==  1: fitness[i] += 1
            # elif res == -1: fitness[j] += 1
            # elif res ==  0: continue#print(f'Tie between {strat1} and {strat2}')
            res = list(score_seeds(list(pop[i]), list(pop[j]), adj_list).values())
            fitness[i] += res[0]
            fitness[j] += res[1]
    return fitness


In [5]:
def select_mating_pool(pop, fitness, num_parents):
    parents = np.zeros((num_parents, num_seeds))
    for parent_num in range(num_parents):
        max_fitness_idx = np.argmax(fitness)
        parents[parent_num] = pop[max_fitness_idx]
        fitness[max_fitness_idx] = -999999999
    return parents

In [6]:
def crossover(parents, offspring_size):
    offspring = np.empty((offspring_size, num_seeds))
    for k in range(offspring_size):
        parent1_idx, parent2_idx = np.random.choice(num_parents, 2, replace=False)
        parents_seeds = np.unique(np.concatenate((parents[parent1_idx], parents[parent2_idx])))
        test = np.random.choice(parents_seeds, num_seeds, replace=False)
        offspring[k] = test
    return offspring

In [7]:
def mutation(offspring_crossover, G, num_mutations):
    for idx in range(offspring_crossover.shape[0]):
        while True:
            choice = np.random.choice(G.nodes, num_mutations, replace=False).astype(int)
            if not np.isin(choice, offspring_crossover[idx]).any():
                offspring_crossover[idx, :num_mutations] = choice
                break
    return offspring_crossover


In [8]:
num_generations = 50
old_parents = np.empty(1)
num_same = 0
for i in tqdm(range(num_generations)):
    fitness = calc_pop_fitness(population, G)
    parents = select_mating_pool(population, fitness, num_parents)
    # if np.array_equiv(parents, old_parents):
    #     num_same += 1
    # else:
    #     num_same = 0
    # if num_same == 5:
    #     break
    old_parents = parents
    offspring = crossover(parents, pop_size-num_parents)
    offspring = mutation(offspring, G, num_mutations)
    population = []
    population.extend(parents.astype(int).astype(str))
    population.extend(offspring.astype(int).astype(str))
fitness = calc_pop_fitness(population, G)
winning_strategy = population[np.argmax(fitness)]

 86%|████████▌ | 43/50 [01:15<00:12,  1.77s/it]


KeyboardInterrupt: 

In [ ]:
other = np.array(['13', '2', '11', '92', '9', '8', '10', '75', '33', '17'])
genetic = winning_strategy
score_seeds(genetic, other, nx.to_dict_of_lists(G))

{'1': 11.0, '2': 186.0}

In [ ]:

# genetic = winning_strategy

[200.   0.   0.]


{'1': 0.0, '2': 0.0}